In [71]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [72]:
# catalog name
catalog = 'groups_088.hdf5'

In [73]:
# value of the scale factor (for redshift 0.05)
scale_factor = 0.95238

# open the catalogue
f = h5py.File(catalog, 'r')

In [74]:
# read the positions, velocities and masses of the FoF halos
pos_h  = f['Group/GroupPos'][:]/1e3           #positions in Mpc/h
vel_h  = f['Group/GroupVel'][:]/scale_factor  #velocities in km/s
mass_h = f['Group/GroupMass'][:]*1e10         #masses in Msun/h

# read the positions, black hole masses and stellar masses of the subhalos/galaxies
pos_g  = f['Subhalo/SubhaloPos'][:]/1e3         #positions in Mpc/h
BH_g   = f['Subhalo/SubhaloBHMass'][:]*1e10     #black-hole masses in Msun/h
M_star = f['Subhalo/SubhaloMassType'][:,4]*1e10 #stellar masses in Msun/h

# close file
f.close()

In [75]:
#find the galaxies
galaxy_pos = pos_g[M_star > 0]
print(str(M_star.shape[0]) + ' subhalos and ' + str(galaxy_pos.shape[0]) + ' galaxies.')

17267 subhalos and 2446 galaxies.


In [78]:
%matplotlib qt
fig = plt.figure(figsize = (8, 6))
ax = fig.add_subplot(projection='3d')
ax.set_xlim(0, 25)
ax.set_ylim(0, 25)
ax.set_zlim(0, 25)
ax.set_xlabel('X [Mpc/h]')
ax.set_ylabel('Y [Mpc/h]')
ax.set_zlabel('Z [Mpc/h]')
ax.scatter(galaxy_pos[:,0], galaxy_pos[:,1], galaxy_pos[:,2], s = 1.5, alpha = 0.7, color = 'red')
plt.tight_layout()
plt.show()

In [ ]:
#check to save